In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0,'/home/michele/lavoro/code/librascal/build/')

# Spherical expansion coefficients

This notebook provides examples of the kind of manipulations that need to be applied to rotate structures and spherical expansion coefficients. First, using traditional complex-spherical-harmonics tools, then, converting those to a fully real-valued pipeline based on a couple of handy utility functions

In [ ]:
from ase.io import read
import ase
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt

from rascal.representations import SphericalExpansion, SphericalInvariants
from rascal.utils import (get_radial_basis_covariance, get_radial_basis_pca, 
                          get_radial_basis_projections, get_optimal_radial_basis_hypers )
from rascal.utils import radial_basis
from rascal.utils import WignerDReal, ClebschGordanReal, spherical_expansion_reshape, lm_slice

In [ ]:
# imports also some internals to demonstrate manually some CG manipulations
from rascal.utils.cg_utils import _r2c as r2c
from rascal.utils.cg_utils import _c2r as c2r
from rascal.utils.cg_utils import _cg as cg
from rascal.utils.cg_utils import _rotation as rotation
from rascal.utils.cg_utils import _wigner_d as wigner_d

## Loads the structures

In [ ]:
import urllib.request
# a collection of distorted ethanol molecules from the ANI-1 dataset 
# (see https://github.com/isayev/ANI1_dataset) with energies and forces computed using DFTB+ 
# (see https://www.dftbplus.org/)
url = 'https://raw.githubusercontent.com/cosmo-epfl/librascal-example-data/833b4336a7daf471e16993158322b3ea807b9d3f/inputs/molecule_conformers_dftb.xyz'
# Download the file from `url`, save it in a temporary directory and get the
# path to it (e.g. '/tmp/tmpb48zma.txt') in the `structures_fn` variable:
structures_fn, headers = urllib.request.urlretrieve(url)
structures_fn

In [ ]:
# Total number of structure to load
N = 100

# load the structures
frames = read(structures_fn,':{}'.format(N))

## Utility functions

Numerical evaluation of rotations and CG coefficients from sympy

In [ ]:
ml1, ml2, mL = 2,3,1

In [ ]:
mcg = cg(ml1, ml2, mL)

In [ ]:
mcg[:,:,2]

## Demonstrate the equivariance of spherical expansion coefficients

first, we compute the density expansion coefficients on a representative dataset

In [ ]:
spherical_expansion_hypers = {
    "interaction_cutoff": 3,
    "max_radial": 8,
    "max_angular": 6,
    "gaussian_sigma_constant": 0.3,
    "gaussian_sigma_type": "Constant",
    "cutoff_smooth_width": 0.5,
    "radial_basis": "GTO",
}

spex = SphericalExpansion(**spherical_expansion_hypers)

In [ ]:
selframe = frames[8]
sell = 3
feats = spex.transform(selframe).get_features(spex)
feats *= 1e6
rfeats = spherical_expansion_reshape(feats, **spherical_expansion_hypers)

In [ ]:
def relayout(hypers, x):
    # reshape spherical harmonics coefficients
    nmax = hypers['max_radial']
    lmax = hypers['max_angular']
    n = len(x)
    nel = x.shape[1]//(nmax*(lmax+1)**2)
    rx = x.reshape((n, nel, nmax, (lmax+1)**2)) # (lm) terms are stored in a compact form    
    return rx

In [ ]:
# random rotation in terms of Euler angles
abc = np.random.uniform(size=(3))*np.pi

In [ ]:
# this is the Cartesian rotation matrix (helper function, ZYZ convention)
mrot = rotation(*abc)

In [ ]:
# rotated structure and associated features
rotframe = selframe.copy()
rotframe.positions = rotframe.positions @ mrot.T
rotframe.cell = rotframe.cell @ mrot.T   # rotate also the cell
rotfeats = spex.transform(rotframe).get_features(spex)
rotfeats *= 1e6
rfeats_rot = spherical_expansion_reshape(rotfeats, **spherical_expansion_hypers)

In [ ]:
print(rfeats[0,0,0,lm_slice(sell)])
np.linalg.norm(rfeats[0,0,0,lm_slice(sell)])

In [ ]:
print(rfeats_rot[0,0,0,lm_slice(sell)])
np.linalg.norm(rfeats_rot[0,0,0,lm_slice(sell)])

## Rotate the spherical expansion features using Wigner matrices

In [ ]:
# computing the wigner matrix takes some time for L>4
mwd = wigner_d(sell, *abc)

In [ ]:
# orthogonality
plt.matshow(np.real(np.conjugate(mwd.T)@mwd))

In [ ]:
#  back and forth to check transformation from real to complex SPH
c2r(r2c(rfeats[0,0,0,lm_slice(sell)])) - rfeats[0,0,0,lm_slice(sell)]

In [ ]:
rfeats[0,0,0,lm_slice(sell)]

In [ ]:
rfeats_rot[0,0,0,lm_slice(sell)]

In [ ]:
c2r(np.conjugate(mwd)@r2c(rfeats[0,0,0,lm_slice(sell)]))

## Compute CG iteration and show that it transforms properly

basically, here we compute covariant, lambda-SOAP features by combining spherical expansion coefficients,
following the idea behind NICE [[original paper](doi.org/10.1063/5.0021116)]

In [ ]:
# these are the indices of the features 
ml1, ml2, mL = 3,2,3
mcg = cg(ml1, ml2, mL)
mwd = wigner_d(mL, *abc)

In [ ]:
cg1 = c2r(np.einsum("abc,a,b->c",mcg,
                    r2c(rfeats[0,0,0,lm_slice(ml1)]), 
                    r2c(rfeats[0,0,0,lm_slice(ml2)])))

In [ ]:
rotcg1 = c2r(np.einsum("abc,a,b->c",mcg,
                    r2c(rfeats_rot[0,0,0,lm_slice(ml1)]), 
                    r2c(rfeats_rot[0,0,0,lm_slice(ml2)])) )

In [ ]:
cg1

In [ ]:
rotcg1

In [ ]:
c2r(np.conjugate(mwd)@r2c(cg1))

## Direct real transformations

There's no "real" reason to go through the complex algebra for rotations - we can transform once and for all the coefficients and be done with that!

In [ ]:
# matrix version of the real-2-complex and complex-2-real transformations
r2c_mat = np.hstack([r2c(np.eye(2*mL+1)[i])[:,np.newaxis] for i in range(2*mL+1)])
c2r_mat = np.conjugate(r2c_mat.T)

In [ ]:
# we can use this to transform features
r2c_mat @ cg1 - r2c(cg1)

In [ ]:
# and Wigner D matrix as well
real_mwd = np.real(c2r_mat @ np.conjugate(mwd) @ r2c_mat)

The direct real rotation is equal (modulo noise) to going back and forth from complex sph

In [ ]:
real_mwd @ cg1 - rotcg1

this also applies to the CG iteration!

In [ ]:
r2c_mat_l1 = np.hstack([r2c(np.eye(2*ml1+1)[i])[:,np.newaxis] for i in range(2*ml1+1)])
r2c_mat_l2 = np.hstack([r2c(np.eye(2*ml2+1)[i])[:,np.newaxis] for i in range(2*ml2+1)])
r2c_mat_L = np.hstack([r2c(np.eye(2*mL+1)[i])[:,np.newaxis] for i in range(2*mL+1)])

In [ ]:
real_mcg = np.real(np.einsum("abc, ax, by, zc -> xyz", mcg, r2c_mat_l1, r2c_mat_l2, np.conjugate(r2c_mat_L.T)))

In [ ]:
real_cg1 = np.einsum("abc,a,b->c",real_mcg,
                    rfeats[0,0,0,lm_slice(ml1)],
                    rfeats[0,0,0,lm_slice(ml2)])

In [ ]:
real_cg1 - cg1

# Streamlined real-only CG iter (and transformation)

Uses the utility classes defined in rascal.utils to do all of the above (and more!)
WignerDReal is a Wigner D matrix implementation to rotate Y^m_l - like coefficients
ClebschGordanReal precomputes Clebsch-Gordan operations using real-only storage of the spherical expansion coefficients

In [ ]:
WD   = WignerDReal(spherical_expansion_hypers["max_angular"], *abc)
CGIR = ClebschGordanReal(spherical_expansion_hypers["max_angular"])

In [ ]:
scale = 1
test_feats = [ rfeats[0,0,0,l**2:(l+1)**2] *scale  for l in range(0,5) ]
test_feats_rot = [ rfeats_rot[0,0,0,l**2:(l+1)**2]*scale  for l in range(0,5) ]

In [ ]:
t1 = CGIR.combine(test_feats[3], test_feats[4], 3)
t1_r = CGIR.combine(test_feats_rot[3], test_feats_rot[4], 3)

In [ ]:
t1

In [ ]:
WD.rotate(t1)

In [ ]:
t1_r

In [ ]:
t2 = CGIR.combine(t1, test_feats[3], 2)
t2_r = CGIR.combine(t1_r, test_feats_rot[3], 2)

In [ ]:
WD.rotate(t2)

In [ ]:
t2_r

In [ ]:
t3 = CGIR.combine(t2, test_feats[3], 1)
t3_r = CGIR.combine(t2_r, test_feats_rot[3], 1)

In [ ]:
WD.rotate(t3)

In [ ]:
t3_r

In [ ]:
t4 = CGIR.combine(t3, t2, 3)
t4_r = CGIR.combine(t3_r, t2_r, 3)

In [ ]:
WD.rotate(t4)

In [ ]:
t4_r

# Bulk calculation of features

the combination can also be done in bulk! This does it combining elementwise (basically computing the diagonal squares of the features)

In [ ]:
t1_bulk = CGIR.combine(rfeats[:,:,:,lm_slice(3)], rfeats[:,:,:,lm_slice(2)], 3)

In [ ]:
t1_bulk.shape

## Manually compute SOAP and check they match librascal's

In [ ]:
def sph_conjugate_real(sp):
    """ Computes the "complex conjugate" of real spherical harmonics or 
    associated coefficients, basically <lm|rhat>^* = (-1)^m <l (-m)|rhat>.
    In this form, the transformation applies also to real coefficients. 
    Assumes that the argument is a m=(-l..l) block. 
    """
    
    lm = sp.shape[-1]    
    l = (lm - 1)//2
    m1m = np.ones(lm)
    if l%2==0:
        m1m[1::2] = -1
    else:
        m1m[0::2] = -1
    return sp[...,::-1]*m1m

In [ ]:
r2c_mats = {}
c2r_mats = {}
for L in range(0, 7 + 1):
    r2c_mats[L] = np.hstack(
        [r2c(np.eye(2 * L + 1)[i])[:, np.newaxis] for i in range(2 * L + 1)]
    )
    c2r_mats[L] = np.conjugate(r2c_mats[L]).T

In [ ]:
soap_hypers = deepcopy(spherical_expansion_hypers)

soap_hypers["soap_type"] = "PowerSpectrum"
soap_hypers["normalize"] = False

soap = SphericalInvariants(**soap_hypers)
soap_feats = soap.transform(selframe).get_features(soap)*1e12  # scale

nel = len(set(selframe.symbols))
elidx = np.triu_indices(nel)

complex-valued calculation of SOAP

In [ ]:
soap_manual_complex_2013 =  np.moveaxis(np.asarray([
    np.real(np.einsum("ianm,iANm->ianAN",
              rfeats[...,lm_slice(l)]@r2c_mats[l], 
              np.conjugate(rfeats[...,lm_slice(l)]@r2c_mats[l])
             ))/np.sqrt(2*l+1)
    for l in range(soap_hypers["max_angular"]+1)]),
    (1,2,4,3,5,0),(0,1,2,3,4,5))[:,elidx[0],elidx[1],...].reshape((len(selframe), -1))    

In [ ]:
soap_manual_real_2013 =  np.moveaxis(np.asarray([
    np.real(np.einsum("ianm,iANm->ianAN",
              rfeats[...,lm_slice(l)], 
              rfeats[...,lm_slice(l)]
             ))/np.sqrt(2*l+1)
    for l in range(soap_hypers["max_angular"]+1)]),
    (1,2,4,3,5,0),(0,1,2,3,4,5))[:,elidx[0],elidx[1],...].reshape((len(selframe), -1))    

In [ ]:
soap_manual_cgreal =  np.moveaxis(np.asarray([
    np.einsum("ianm,iANm->ianAN",
              rfeats[...,lm_slice(l)], 
              sph_conjugate_real(rfeats[...,lm_slice(l)])
             )/np.sqrt(2*l+1)
    for l in range(soap_hypers["max_angular"]+1)]),
    (1,2,4,3,5,0),(0,1,2,3,4,5))[:,elidx[0],elidx[1],...].reshape((len(selframe), -1))    

In [ ]:
tsoap = np.moveaxis(np.asarray([
 CGIR.combine_einsum(rfeats[:,:,:,lm_slice(l)], rfeats[:,:,:,lm_slice(l)], 0, 
                     "ian,iAN->ianAN")
 for l in range(soap_hypers["max_angular"]+1)  
]),(1,2,4,3,5,0,6),(0,1,2,3,4,5,6))[:,elidx[0],elidx[1],...].reshape((len(selframe), -1))

In [ ]:
soap_feats[0,:20]

In [ ]:
soap_manual_complex_2013[0, :20]

In [ ]:
soap_manual_real_2013[0, :20]

In [ ]:
soap_manual_cgreal[0, :20]

In [ ]:
tsoap[0,:20]

In [ ]:
np.linalg.norm(
np.conjugate(rfeats[...,lm_slice(1)]@r2c_mats[1])@c2r_mats[1] - sph_conjugate_real(rfeats[...,lm_slice(1)])
)

In [ ]:
rfeats[...,lm_slice(2)]

In [ ]:
sph_conjugate_real(rfeats[...,lm_slice(2)])

## $\lambda$-SOAP

this instead uses a more general core iteration based on einsum. This below is basically lambda-SOAP

In [ ]:
t1_eins = CGIR.combine_einsum(rfeats[:,:,:,lm_slice(3)], rfeats[:,:,:,lm_slice(2)], 3, "ian,iAN->ianAN")

In [ ]:
t1_eins.shape

Want $\lambda$-SOAP? You've got it! This computes the full set of features (many will be zeros but oh well) using CG iteration. Input spherical harmonics should be already "unrolled" into a n_env, n_el, n_max, (lmax+1)^2 form.
Result is also returned fully unrolled, the first index being the environment, and the last the lambda-SOAP mu index.

*Note this may differ from existing definitions because of the scaling of coefficients*

In [ ]:
def lambda_soap(spx, lam, cg):
    lmax = int(np.sqrt(spx.shape[-1]))-1
    nid, nel, nmax = spx.shape[:-1]
    lsoap = np.zeros((nid, nel, nmax, lmax+1, nel, nmax, lmax+1, 2*lam+1))
    for l1 in range(lmax+1):
        for l2 in range(lmax+1):            
            lsoap[:,:,:,l1,:,:,l2] = cg.combine_einsum(spx[..., lm_slice(l1)],
                                        spx[..., lm_slice(l2)], 
                                        lam, combination_string="ian,iAN->ianAN")
    return lsoap

In [ ]:
%%time
lsoap = lambda_soap(rfeats, 3, CGIR)
lsoap.shape = (lsoap.shape[0], -1, lsoap.shape[-1])
lsoap_rot = lambda_soap(rfeats_rot, 3, CGIR)
lsoap_rot.shape = (lsoap_rot.shape[0], -1, lsoap_rot.shape[-1])

In [ ]:
lsoap[0,101]

In [ ]:
WD.rotate(lsoap[0,101])

In [ ]:
lsoap_rot[0,101]

# Products of features

This is useful to transform quantities that can be construed as products of spherical harmonics to a coupled form, and back. That is, if you have Y^m1_l1 Y^m2_l2 you can cast it into a series of coefficients that transform like a single Y^M_L, and back. Note that the transformation depends on the initial values of l1,l2

In [ ]:
scale = 1e0
test_feats = [ rfeats[0,0,0,lm_slice(l)] *scale  for l in range(0,5) ]
test_feats_rot = [ rfeats_rot[0,0,0,lm_slice(l)]*scale  for l in range(0,5) ]

test_prod = test_feats[2][:,np.newaxis]@test_feats[3][np.newaxis,:]
test_prod_rot = test_feats_rot[2][:,np.newaxis]@test_feats_rot[3][np.newaxis,:]

In [ ]:
test_coupled = CGIR.couple(test_prod)

In [ ]:
test_coupled[1]

In [ ]:
test_coupled_rot = CGIR.couple(test_prod_rot)

In [ ]:
test_coupled_rot[1][3]

In [ ]:
WD.rotate(test_coupled[1][3])

In [ ]:
test_decoupled = CGIR.decouple(test_coupled)

In [ ]:
test_prod - test_decoupled

In [ ]:
l1, l2 = 2, 3

this can be shown by seeing that the real CG are orthogonal

In [ ]:
prod = np.zeros((2*l1+1,2*l2+1,2*l1+1,2*l2+1))
for L in range(abs(l1-l2), abs(l1+l2)+1):
    for M in range(0, 2*L+1):
        for m1, m2, mcg in CGIR._cgdict[(l1, l2, L)][M]:
            for m1p, m2p, mcgp in CGIR._cgdict[(l1, l2, L)][M]:
                prod[m1,m2,m1p,m2p] += mcg*mcgp

In [ ]:
pr = prod.reshape((2*l1+1)*(2*l2+1),(2*l1+1)*(2*l2+1))
plt.matshow(pr)